# 제4 고지 : 신경망 만들기 
## STEP 49 : Dataset 클래스와 전처리

이전에 spiral dataset을 가져올 때는 데이터의 갯수가 작았으므로, `ndarray` 하나로 표현할 수 있었지만, 대규모 데이터셋의 경우에는 모든 데이터를 메모리에 올릴 수 없다. 그래서 이를 대응하기 위해 `Dataset` 클래스를 만들어 데이터를 처리할 수 있도록 한다.


### 49.1 Dataset 클래스 구현

`Dataset` 클래스는 기반 클래스로서의 역할을 하고, 사용자가 실제로 사용하는 데이터셋은 이를 상속하여 구현한다.

```python
class Dataset:
    def __init__(self,train=True):
        self.train = train
        self.data = None
        self.label = None
        self.prepare() 

    def __getitem__(self,index):
        assert np.isscalar(index) # index 는 스칼라(정수)만 가능
        if self.label is None:
            return self.data[index],None
        else:
            return self.data[index],self.label[index]
    
    def __len__(self):
        return len(self.data)

    def prepare(self):
        pass
```

여기서 주목해야 할것은 `__getitem__` 과 `__len__` 메서드로, `x[0]` 과 같이 인덱스로 접근가능하며 `len()` 함수를 호출하여 데이터셋의 크기를 알 수 있다는 것이다. 

이제 이를 상속하여 spiral dataset 을 구현하면 다음과 같다.

```python
class Spiral(Dataset):
    def prepare(self):
        self.data,self.label = get_spiral(self.train)
```






In [1]:
import sys
sys.path.append("..")

import dezero

train_set = dezero.datasets.Spiral(train=True)
print(train_set[0])
print(len(train_set))

(array([-0.13981389, -0.00721657], dtype=float32), 1)
300


### 49.2 큰 데이터셋의 경우

현재는 작은 데이터셋에는 문제 없지만, 데이터셋이 훨씬 크다면 이런 방식은 사용핦 수 없다. 그래서 다음과 같은 구현방식을 생각해본다.

1. `data` `label` 디렉토리에 각각 100만개의 데이터가 저장 (ex, `data/0.npy`, `label/1.npy`)
2. `Dataset` 클래스를 초기화할때 데이터를 불러오는 것이 아닌 데이터에 접근할 때 데이터를 읽게 하는 것이다.

```python
class BigData(dataset):
    def __getitem__(index):
        x = np.load("data/{}.npy".format(index))
        t = np.load("label/{}.npy".format(index))
        return 
```

### 49.3 데이터 이어 붙이기

신경망을 학습 시킬 때 데이터셋 중 일부를 미니배치로 가져온다. 

```python
train_set = dezero.datasets.Spiral()
batch_index = [0,1,2] 
batch = [train_set[i] for i in batch_index] 
# batch = [(data_0,label_0),(data_1,label_1),(data_2,label_2)]

x = np.array([example[0] for example in batch])
t = np.array([example[1] for example in batch])

print(x.shape)
# (3,2)
print(t.shape)
# (3,)
```

이와 같이 우선은 인덱스를 가져와 여러 데이터를 꺼내고 이 후 DeZero의 신경망에 입력하기 위해 다음과 같이 하나의 `ndarray` 인스턴스로 변환하여 이어붙인다.


### 49.4 학습 코드
그럼 이제 `Spiral` 클래스를 사용하여 학습을 진행해본다.


In [2]:
import sys
sys.path.append("..")

import math
import numpy as np
import dezero
from dezero import optimizers
import dezero.functions as F
from dezero.models import MLP


log_interval = 20 # 20 epoch 마다 logging

# 1. 하이퍼 파라미터 설정
max_epoch = 300
batch_size = 30
hidden_size = 10
lr = 1.0


# 2. 데이터 읽기 / 모델, 옵티마이저 생성
######################################
train_set = dezero.datasets.Spiral()
######################################

model = MLP((hidden_size, 3))
optimizer = optimizers.SGD(lr).setup(model)

######################################
data_size = len(train_set)
######################################

max_iter = math.ceil(data_size / batch_size)

for epoch in range(max_epoch):
    # 3. 데이터셋 셔플
    index = np.random.permutation(data_size)
    sum_loss = 0

    for i in range(max_iter):
        # 4. 미니 배치 생성
        batch_index = index[i * batch_size:(i + 1) * batch_size]
        #########################################################
        batch = [train_set[i] for i in batch_index] 
        batch_x = np.array([example[0] for example in batch])
        batch_t = np.array([example[1] for example in batch])
        #########################################################
        # 5. 기울기 산출 / 매개변수 갱신
        y = model(batch_x)
        loss = F.softmax_cross_entropy(y, batch_t)
        model.cleargrads()
        loss.backward()
        optimizer.update()

        sum_loss += float(loss.data) * len(batch_t)

    # 6. 에폭마다 학습 경과 출력
    avg_loss = sum_loss / data_size
    if epoch % log_interval == 0:
        print('epoch %d, loss %.2f' % (epoch + 1, avg_loss))


epoch 1, loss 1.13
epoch 21, loss 0.73
epoch 41, loss 0.71
epoch 61, loss 0.64
epoch 81, loss 0.52
epoch 101, loss 0.42
epoch 121, loss 0.36
epoch 141, loss 0.27
epoch 161, loss 0.23
epoch 181, loss 0.20
epoch 201, loss 0.17
epoch 221, loss 0.16
epoch 241, loss 0.15
epoch 261, loss 0.15
epoch 281, loss 0.13


### 49.5 데이터 전처리

머신러닝에서는 모델에 데이터를 입력하기 전에 데이터를 특정한 형태로 가공하는 일이 많다.  
예를 들어 **데이터에서 특정 값을 제거하거나 형상을 변환**하는 것이다. 이미지 인식에서는 **데이터 증강(data augmentation)** 을 위해 자주 사용되기도 한다. 따라서 해당 기능을 추가해본다.

```python
class Dataset:
    def __init__(self, train=True, transform=None, target_transform=None):
        self.train = train
        ##################################################
        self.transform = transform
        self.target_transform = target_transform
        if self.transform is None:
            self.transform = lambda x: x
        if self.target_transform is None:
            self.target_transform = lambda x: x
        ##################################################

        self.data = None
        self.label = None
        self.prepare()

    def __getitem__(self, index):
        assert np.isscalar(index)
        ##################################################
        if self.label is None:
            return self.transform(self.data[index]), None
        else:
            return self.transform(self.data[index]),\
                   self.target_transform(self.label[index])
        ##################################################
    def __len__(self):
        return len(self.data)

    def prepare(self):
        pass

```

구체적으로 살펴보면 `transform` 과 `target_transform` 은 각각 데이터, 레이블에 대핸 변환을 수행하는 `Callable` 객체이다. 만약 정의된 것이 없다면 데이터,레이블을 그대로 반환하도록 한다.

<span style='background-color : #ffdce0'>💡<b>(`dezero/transforms.py` 전처리시 자주 사용되는 변환들이 구현되어 있다.) </b></span>


In [5]:
from dezero import transforms

f = transforms.Normalize(mean=0.0,std=2.0)
train_set = dezero.datasets.Spiral(transform=f)

f = transforms.Compose([transforms.Normalize(mean=0.0,std=2.0),transforms.AsType(np.float64)])
train_set = dezero.datasets.Spiral(transform=f)